# 数据对接

In [4]:
%%time
# coding=utf8
import pymysql
import pymssql 
import numpy as np
import pandas as pd
import pickle
import redis
import datetime as dtt
# from cma.music.DataQueryClient import DataQueryClient
import json
import geojsoncontour
from scipy.interpolate import griddata
from scipy.interpolate import interp1d
from astropy.convolution import convolve, Gaussian2DKernel, Tophat2DKernel
from astropy.modeling.models import Gaussian2D
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import xarray as xr
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

class nlcmap(LinearSegmentedColormap):
    """A nonlinear colormap"""

    name = 'nlcmap'

    def __init__(self, cmap, levels):
        self.cmap = cmap
        self.monochrome = self.cmap.monochrome
        self.levels = np.asarray(levels, dtype='float64')
        self._x = self.levels/ self.levels.max()
        self.levmax = self.levels.max()
        self.levmin = self.levels.min()
        self._y = np.linspace(self.levmin, self.levmax, len(self.levels))

    def __call__(self, xi, alpha=1.0, **kw):
        yi = np.interp(xi, self._x, self._y)
        return self.cmap(yi/self.levmax, alpha)

        
# 气象快报 
class station_text:
    def __init__(self,city_code,start,end):
        #self.rs = redis.Redis(host='127.0.0.1', port=6379)
#         self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="tzqxj58660",db="ZJSZDZDB")
        self.userId = "BEHZ_TZSJ_TZSERVICE" 
        self.pwd = "Liyuan3970!@" 
        self.dataFormat = "json"
        self.city_code = city_code
        self.start = start
        self.end = end
        self.data = self.comupt_city_csv(city_code,start,end)
        self.city_codes = ["331000"]
    def comput_city(self,city_code,start,end):
        """快报或者统计数据的接口"""
        # getSurfEleInRegionByTime
        date_start = dtt.datetime.strptime(start,'%Y-%m-%d %H:%M:%S') 
        date_end = dtt.datetime.strptime(end,'%Y-%m-%d %H:%M:%S') 
        offset = dtt.timedelta(minutes=-60*8)
        label_start = (date_start + offset).strftime('%Y%m%d%H%M') + "00"
        label_end = (date_end + offset).strftime('%Y%m%d%H%M') + "00"   
        labels = "[" + label_start + "," + label_end + "]"
        #timelabel = self.decode_time_str(timesdelay)
        client = DataQueryClient()
        interfaceId = "statSurfEleInRegion"  
        params = {
            'dataCode':"SURF_CHN_MUL_MIN",  #SURF_CHN_MUL_HOR
            'adminCodes':"331000",
            'elements':"Province,City,Cnty,Town,Lat,Lon,Alti,Station_levl,Station_Name,Station_Id_C,",
            'timeRange':labels,
            'statEles':'MAX_TEM,MIN_TEM,SUM_PRE,MAX_PRE_1h,MIN_VIS_HOR_1MI,MAX_WIN_S_Gust_Max,MAX_WIN_S_Avg_2mi',
            #'orderBy':"MAX_WIN_S_Inst_Max:desc",
            'staLevels':"011,012,013,014,015,016", # 12国家站 14区域站
            'limitCnt':"100000000"
        }
        result = client.callAPI_to_serializedStr(self.userId, self.pwd, interfaceId, params, self.dataFormat)
        #rint(result)
        result_json = json.loads(result)
        clomns = "Province,City,Cnty,Town,Lat,Lon,Alti,Station_levl,Station_Name,Station_Id_C,MAX_TEM,MIN_TEM,SUM_PRE,MAX_PRE_1h,MIN_VIS_HOR_1MI,MAX_WIN_S_Gust_Max,MAX_WIN_S_Avg_2mi".split(",")
        data = pd.DataFrame(result_json['DS'],columns=clomns)
        data.columns = "Province,City,Cnty,Town,Lat,Lon,Height,Stationlevl,StationName,IIIII,tmax,tmin,rain,rainhour,view,wind,windave".split(",")
        return data
    def comupt_city_csv(self,city_code,start,end):
#         data = pd.read_csv("static/data/downfile/comput.csv")
        data = pd.read_csv("downfile/comput.csv")
        return data
    def text_wind(self,plot_data):
        if plot_data=="none":
            orig = self.data
            data = orig.sort_values(by="wind",ascending=False)#.head(5).to_dict()
            ## 数据分级
            bins=[0,10.7,13.8,17.1,20.7,24.4,28.4,32.6,36.9,41.4,46.1,50.9,56,80]
            labels=['6级风以下','6级风','7级风','8级风','9级风','10级风','11级风','12级风','13级风','14级风','15级风','16级风','17级风']
            wind = data[(data['wind']>10.7) & (data['wind']<5009)]
            wind['rank']=pd.cut(wind['wind'],bins,right=False,labels=labels)
            del wind['Unnamed: 0'] # 天擎不需要
            wind.reset_index(drop=True)# 天擎不需要
        else:
            wind = pd.read_json(json.dumps(plot_data), orient='records')
        # 获取单站较大的有
        testframe = wind.head(5)
        wind_json = wind.sort_values(by="wind",ascending=False).to_json(orient='records',force_ascii=False) 
        indextext = ""
        for index,row in testframe.iterrows():
            if self.city_code in self.city_codes:
                indextext = indextext + row['Cnty'] + row['StationName'] + str(row['wind']) + "米/秒，"
                #print(index,row['City'],row['Cnty'],row['Town'],row['wind'])
            else:
                indextext = indextext + row['Town'] + row['StationName'] + str(row['wind']) + "米/秒，"
                #print(index,row['City'],row['Cnty'],row['Town'],row['wind'])
        indextext = indextext[:-1] + "。"
        rank_station = {
            '17级及以上风':len(wind[(wind['wind']>=56.4) & (wind['wind']<80)].value_counts()),
            '16级风':len(wind[(wind['wind']>=50.9) & (wind['wind']<56.4)].value_counts()),
            '15级风':len(wind[(wind['wind']>=46.1) & (wind['wind']<50.9)].value_counts()),
            '14级风':len(wind[(wind['wind']>=41.4) & (wind['wind']<46.1)].value_counts()),
            '13级风':len(wind[(wind['wind']>=36.9) & (wind['wind']<41.4)].value_counts()),
            '12级风':len(wind[(wind['wind']>=32.6) & (wind['wind']<36.9)].value_counts()),
            '11级风':len(wind[(wind['wind']>=28.4) & (wind['wind']<32.6)].value_counts()),
            '10级风':len(wind[(wind['wind']>=24.4) & (wind['wind']<28.4)].value_counts()),
            '9级风':len(wind[(wind['wind']>=20.7) & (wind['wind']<24.4)].value_counts()),
            '8级风':len(wind[(wind['wind']>=17.1) & (wind['wind']<20.7)].value_counts()),
            '7级风':len(wind[(wind['wind']>=13.8) & (wind['wind']<17.1)].value_counts()),
            '6级风':len(wind[(wind['wind']>=10.7) & (wind['wind']<13.8)].value_counts()),
            '6级风以下':len(wind[(wind['wind']>0) & (wind['wind']<10.7)].to_dict())
        }
        #  统计各等级风力的个数
        numbertext = ""
        for key, value in rank_station.items():
            if value>0:
                numbertext = numbertext + key +"有" + str(value) + "站,"
        numbertext = numbertext[:-1] + "。"
        if rank_station['17级及以上风'] >0 :
            text = "【风力通报】 全市出现17级及以上大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['16级风'] >0 :
            text = "【风力通报】 全市出现16级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext  
        elif rank_station['15级风'] >0 :
            text = "【风力通报】 全市出现15级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['14级风'] >0 :
            text = "【风力通报】 全市出现14级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['13级风'] >0 :
            text = "【风力通报】 全市出现13级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['12级风'] >0 :
            text = "【风力通报】 全市出现12级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['11级风'] >0 :
            text = "【风力通报】 全市出现11级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['10级风'] >0 :
            text = "【风力通报】 全市出现10级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['9级风'] >0 :
            text = "【风力通报】 全市出现7～9级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['8级风'] >0 :
            text = "【风力通报】 全市出现6～8级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        else:
            text = ""
        return text,wind_json
    def text_view(self,plot_data):
        if plot_data=="none":
            orig = self.data
            data = orig.sort_values(by="view",ascending=True)#.head(5).to_dict()
            bins=[0,50,200,500,99999]
            labels=['强浓雾','浓雾','大雾','正常']
            view = data[(data['view']>0) & (data['view']<1000)]
            view['rank']=pd.cut(view['view'],bins,right=False,labels=labels)
            del view['Unnamed: 0'] # 天擎不需要
            view.reset_index(drop=True)# 天擎不需要
        else:
            view = pd.read_json(json.dumps(plot_data), orient='records')
        rank_station = {
           '强浓雾':len(view[(view['view']>0) & (view['view']<50)].to_dict()),
            '浓雾':len(view[(view['view']>=50) & (view['view']<200)].value_counts()),
            '大雾':len(view[(view['view']>=200) & (view['view']<500)].value_counts()),
            '正常':len(view[(view['view']>=500) & (view['view']<99990)].value_counts()) 
        }
        # 获取单站较大的有
        testframe = view.head(5) 
        view_json = view.sort_values(by="view",ascending=True).to_json(orient='records',force_ascii=False)
        indextext = ""
        for index,row in testframe.iterrows():
            if self.city_code in self.city_codes:
                indextext = indextext + row['Cnty'] + row['StationName'] + str(row['view']) + "米，"
            else:
                indextext = indextext + row['Town'] + row['StationName'] + str(row['view']) + "米，"
        indextext = indextext[:-1] + "。"
        #  统计各等级风力的个数
        numbertext = ""
        for key, value in rank_station.items():
            if value>0 and key!="正常":
                numbertext = numbertext + key +"的有" + str(value) + "站,"
        numbertext = numbertext[:-1] + "。"
        if rank_station['强浓雾'] >0 :
            text = "【能见度】 全市出现低于50米的强浓雾，能见度较低的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['浓雾'] >0 :
            text = "【能见度】 全市出现低于200米的浓雾，能见度较低的有："
            text = text + indextext + "其中，"  + numbertext  
        elif rank_station['大雾'] >0 :
            text = "【能见度】 全市出现低于500米的大雾，能见度较低的有："
            text = text + indextext + "其中，"  + numbertext
        else:
            text =""
        return text,view_json
    def text_tmax(self,plot_data):
        if plot_data=="none":
            orig = self.data
            data = orig.sort_values(by="tmax",ascending=False)#.head(5).to_dict()
            bins=[-40,35,37,40,80]
            labels=['正常','35度以上','37度以上','40度以上']
            tmax = data[(data['tmax']>-40) & (data['tmax']<100)]
            tmax['rank']=pd.cut(tmax['tmax'],bins,right=True,labels=labels)
            del tmax['Unnamed: 0'] # 天擎不需要
            tmax.reset_index(drop=True)# 天擎不需要
        else:
            tmax = pd.read_json(json.dumps(plot_data), orient='records')
        rank_station = {
            '40度以上':len(tmax[(tmax['tmax']>=40) & (tmax['tmax']<80)].value_counts()),
            '37度以上':len(tmax[(tmax['tmax']>=37) & (tmax['tmax']<40)].value_counts()),
            '35度以上':len(tmax[(tmax['tmax']>=35) & (tmax['tmax']<37)].value_counts()),
           '正常':len(tmax[(tmax['tmax']>-40) & (tmax['tmax']<35)].to_dict())      
        }
        # 获取单站较大的有
        testframe = tmax.head(5) 
        tmax_json = tmax.sort_values(by="tmax",ascending=False).to_json(orient='records',force_ascii=False)
        indextext = ""
        for index,row in testframe.iterrows():
            if self.city_code in self.city_codes:
                indextext = indextext + row['Cnty'] + row['StationName'] + str(row['tmax']) + "℃，"
            else:
                indextext = indextext + row['Town'] + row['StationName'] + str(row['tmin']) + "℃，"
        indextext = indextext[:-1] + "。"
        #  统计各等级风力的个数
        numbertext = ""
        for key, value in rank_station.items():
            if value>0 and key not in ["正常","35度以上"]:
                numbertext = numbertext + key +"的有" + str(value) + "站,"
        numbertext = numbertext[:-1] + "。"
        if rank_station['40度以上'] >0 :
            text = "【高温通报】 全市出现40℃以上的高温，温度较高的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['37度以上'] >0 :
            text = "【高温通报】 全市出现40℃以上的高温，温度较高的有："
            text = text + indextext + "其中，"  + numbertext  
        elif rank_station['35度以上'] >0 :
            text = "【高温通报】 全市出现40℃以上的高温，温度较高的有："
            text = text + indextext 
        else:
            text = ""
        return text,tmax_json
    def text_rain(self,plot_data):
            # 面雨量
        if plot_data=="none":
            orig = self.data
            data = orig.sort_values(by="rain",ascending=False)
            bins=[0,30,50,80,100,150,200,250,300,400,500,800,1000,2000]
            labels=['≥0毫米','≥30毫米','≥50毫米','≥80毫米','≥100毫米','≥150毫米','≥200毫米','≥250毫米','≥300毫米','≥400毫米','≥500毫米','≥800毫米','≥1000毫米']
            bins_text = [0,5,16.9,37.9,74.5,5000]
            labels_text = ['小雨','小到中雨','中到大雨','大到暴雨','大暴雨']
            rain = data[(data['rain']>0) & (data['rain']<5009)]
            rain['rank']=pd.cut(rain['rain'],bins,right=False,labels=labels)
            rain['rank_label']=pd.cut(rain['rain'],bins_text,right=False,labels=labels_text)
            del rain['Unnamed: 0'] # 天擎不需要
            rain.reset_index(drop=True)
        else:
            rain = pd.read_json(json.dumps(plot_data), orient='records')
        # 数据校验
        def vaild_rain(col):
            lat = col['Lat']
            lon = col['Lon']
            pre = col['rain']
            IIIII = col['IIIII']
            s = lat - 0.07
            n = lat + 0.07
            w = lon - 0.07
            e = lon + 0.07
            stdf = rain[(rain['Lat']>s)&(rain['Lat']<n)&(rain['Lon']<e)&(rain['Lon']>w)]
            stdq = stdf[(True^stdf['IIIII'].isin([IIIII]))]
            stdmean = stdq['rain'].mean()
            stds = stdq['rain'].std()
            stdabs = abs(pre - stdmean)
            if stdabs > stds*10:
                #value = stdabs - stds
                value = 1
            else:
                #value = stdabs - stds
                value = 0
            del stdf,stdq
            return value
        rain['std'] = rain.apply(vaild_rain,axis=1) 
        # 面雨量
        if self.city_code in self.city_codes:
            cnty = rain.groupby(['Cnty'])['rain'].mean().to_dict()
            
            text_average_city = "面雨量：" + str(round(rain['rain'].mean(),2)) + "毫米，" +"各市（县）面雨量如下:"
        else:
            cnty = rain.groupby(['Town'])['rain'].mean().to_dict()
            text_average_city = "各乡镇面雨量如下:"
        items = sorted(cnty.items())
        sorted_cnty = {k: v for k, v in sorted(cnty.items(), key=lambda x: x[1], reverse=True)}
        for key, value in sorted_cnty.items():
            text_average_city = text_average_city + key + ":" + str(round(value,2)) + "毫米,"
        text_average_city = text_average_city[:-1] + "。"    
        # 单站前十
        rain_max = rain.sort_values(by="rain",ascending=False).head(10)
        rain_json = rain.sort_values(by="rain",ascending=False).to_json(orient='records',force_ascii=False)
        indextext = "单站雨量较大的有："
        for index,row in rain_max.iterrows():
            if self.city_code in self.city_codes:
                indextext = indextext + row['Cnty'] + row['StationName'] + str(row['rain']) + "毫米，"
            else:
                indextext = indextext + row['Town'] + row['StationName'] + str(row['rain']) + "毫米，"
        indextext = indextext[:-1] + "。"
        # 乡镇前十
        town_max = rain.groupby(['Town','Cnty'])['rain'].max().sort_values(ascending=False).head(10)
        indextext_town = "乡镇雨量较大的有："
        for index,row in rain_max.iterrows():
            if self.city_code in self.city_codes:
                indextext_town = indextext_town + row['Cnty'] + row['StationName'] + str(row['rain']) + "毫米，"
            else:
                indextext_town = indextext_town + row['Town'] + row['StationName'] + str(row['rain']) + "毫米，"
        indextext_town = indextext_town[:-1] + "。"
        # 雨量个数统计
        rank_station = {
            '≥1000毫米':len(rain[(rain['rain']>=1000) & (rain['rain']<2000)].value_counts()),
            '≥800毫米':len(rain[(rain['rain']>=800) & (rain['rain']<1000)].value_counts()),
            '≥500毫米':len(rain[(rain['rain']>=500) & (rain['rain']<800)].value_counts()),
            '≥400毫米':len(rain[(rain['rain']>=400) & (rain['rain']<500)].value_counts()),
            '≥300毫米':len(rain[(rain['rain']>=300) & (rain['rain']<400)].value_counts()),
            '≥250毫米':len(rain[(rain['rain']>=250) & (rain['rain']<300)].value_counts()),
            '≥200毫米':len(rain[(rain['rain']>=200) & (rain['rain']<250)].value_counts()),
            '≥150毫米':len(rain[(rain['rain']>=150) & (rain['rain']<200)].value_counts()),
            '≥100毫米':len(rain[(rain['rain']>=100) & (rain['rain']<150)].value_counts()),
            '≥80毫米':len(rain[(rain['rain']>=80) & (rain['rain']<100)].value_counts()),
            '≥50毫米':len(rain[(rain['rain']>=50) & (rain['rain']<80)].value_counts()),
            '≥30毫米':len(rain[(rain['rain']>=30) & (rain['rain']<50)].value_counts()),
            '≥10毫米':len(rain[(rain['rain']>=10) & (rain['rain']<30)].value_counts()),
            '≥0毫米':len(rain[(rain['rain']>0) & (rain['rain']<10)].value_counts()),
        }
        numbertext = "其中，"
        for key, value in rank_station.items():
            if value>0 and key not in ["≥0毫米"]:
                numbertext = numbertext + key +"的有" + str(value) + "站,"
        numbertext = numbertext[:-1] + "。"
        # 雨量乡镇数统计
        town_group = rain.groupby(['Town'])['rain'].max()  
        dict_town = {'Town':town_group.index,'rain':town_group.values}
        town_range = pd.DataFrame(data =dict_town)
        town_range['rank']=pd.cut(town_range['rain'],bins,right=False,labels=labels)
        rank_station_town = {
            '≥1000毫米':len(town_range[(town_range['rain']>=1000) & (town_range['rain']<2000)].value_counts()),
            '≥800毫米':len(town_range[(town_range['rain']>=800) & (town_range['rain']<1000)].value_counts()),
            '≥500毫米':len(town_range[(town_range['rain']>=500) & (town_range['rain']<800)].value_counts()),
            '≥400毫米':len(town_range[(town_range['rain']>=400) & (town_range['rain']<500)].value_counts()),
            '≥300毫米':len(town_range[(town_range['rain']>=300) & (town_range['rain']<400)].value_counts()),
            '≥250毫米':len(town_range[(town_range['rain']>=250) & (town_range['rain']<300)].value_counts()),
            '≥200毫米':len(town_range[(town_range['rain']>=200) & (town_range['rain']<250)].value_counts()),
            '≥150毫米':len(town_range[(town_range['rain']>=150) & (town_range['rain']<200)].value_counts()),
            '≥100毫米':len(town_range[(town_range['rain']>=100) & (town_range['rain']<150)].value_counts()),
            '≥80毫米':len(town_range[(town_range['rain']>=80) & (town_range['rain']<100)].value_counts()),
            '≥50毫米':len(town_range[(town_range['rain']>=50) & (town_range['rain']<80)].value_counts()),
            '≥30毫米':len(town_range[(town_range['rain']>=30) & (town_range['rain']<50)].value_counts()),
            '≥10毫米':len(town_range[(town_range['rain']>=10) & (town_range['rain']<30)].value_counts()),
            '≥0毫米':len(town_range[(town_range['rain']>0) & (town_range['rain']<10)].value_counts()),
        }
        numbertext_town = "其中，"
        for key, value in rank_station_town.items():
            if value>0 and key not in ["≥0毫米"]:
                numbertext_town = numbertext_town + key +"的有" + str(value) + "站,"
        numbertext_town = numbertext_town[:-1] + "。"
        # 统计全市雨情
        rank_text = {
            '大暴雨':sum(rain[rain['rank_label']=='大暴雨']['rain']),
            '大到暴雨':sum(rain[rain['rank_label']=='大到暴雨']['rain']),
            '中到大雨':sum(rain[rain['rank_label']=='中到大雨']['rain']),
            '小到中雨':sum(rain[rain['rank_label']=='小到中雨']['rain']),
            '小雨':sum(rain[rain['rank_label']=='小雨']['rain'])
        }
        res = max(rank_text, key=lambda x: rank_text[x])
        if max(rank_text, key=lambda x: rank_text[x])=="大暴雨":
            text = "【雨情通报】 ：全市出现" + res +"。" + text_average_city + indextext + numbertext + indextext_town + numbertext_town
        elif max(rank_text, key=lambda x: rank_text[x])=="大到暴雨":
            text = "【雨情通报】 ：全市出现" + res +"。" + text_average_city + indextext + numbertext + indextext_town + numbertext_town
        elif max(rank_text, key=lambda x: rank_text[x])=="中到大雨":
            text = "【雨情通报】 ：全市出现" + res +"。" + text_average_city + indextext + numbertext + indextext_town + numbertext_town
        elif max(rank_text, key=lambda x: rank_text[x])=="小到中雨":
            text = "【雨情通报】 ：全市出现" + res +"。" + text_average_city + indextext + numbertext + indextext_town + numbertext_town
        elif max(rank_text, key=lambda x: rank_text[x])=="小雨":
            text = "【雨情通报】 ：全市出现" + res +"。" + text_average_city + indextext + numbertext + indextext_town + numbertext_town
        return text,rain_json
    def main(self):
        text = ""
        plot_data = "none"
        text_rain,rain_json = self.text_rain(plot_data)
        text_wind,wind_json = self.text_wind(plot_data)
        text_tmax,tmax_json = self.text_tmax(plot_data)
        text_view,view_json = self.text_view(plot_data)
        text = text + text_rain + "<br>" + text_wind + "<br>" + text_tmax + "<br>" + text_view
        # text = text + text_rain + "\n" + text_wind + "/n" + text_tmax + "/n" + text_view
        return text,rain_json,wind_json,tmax_json,view_json
    def remain(self,plot_rain,plot_wind,plot_tmax,plot_view):
        text = ""
        text_rain,rain_json = self.text_rain(plot_rain)
        text_wind,wind_json = self.text_wind(plot_wind)
        text_tmax,tmax_json = self.text_tmax(plot_tmax)
        text_view,view_json = self.text_view(plot_view)
        text = text + text_rain + "<br>" + text_wind + "<br>" + text_tmax + "<br>" + text_view
        return text,rain_json,wind_json,tmax_json,view_json
    def plot_rain(self,plot_type,plot_data):
        if plot_type =="none":
            orig = self.data
            data = orig.sort_values(by="rain",ascending=False)
            rain = data[(data['rain']>0) & (data['rain']<5009)]
        else:  
            rain = pd.read_json(json.dumps(plot_data), orient='records')
        lat = np.array(rain['Lat'].to_list())
        lon = np.array(rain['Lon'].to_list())
        Zi = np.array(rain['rain'].to_list())
        data_max = max(Zi)
        data_min = min(Zi)
        np.set_printoptions(precision = 2)
        x = np.arange(120.0,122.0,0.015)
        y = np.arange(27.8,29.5,0.015)
        nx0 =len(x)
        ny0 =len(y)
        X, Y = np.meshgrid(x, y)#100*100
        P = np.array([X.flatten(), Y.flatten() ]).transpose()    
        Pi =  np.array([lon, lat ]).transpose()
        Z_linear = griddata(Pi, Zi, P, method = "nearest").reshape([ny0,nx0])
        gauss_kernel = Gaussian2DKernel(0.1)
        smoothed_data_gauss = convolve(Z_linear, gauss_kernel)
        data_xr = xr.DataArray(Z_linear, coords=[ y,x], dims=["lat", "lon"])
        lat = data_xr.lat
        lon = data_xr.lon
        lons, lats = np.meshgrid(lon, lat)
        colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 24降水
        levels = [0,1,10,25,50,100,250,1000]
        cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
        cmap_nonlin = nlcmap(cmaps, levels)
        contour = plt.contourf(lons,lats,data_xr,cmap=cmap_nonlin,levels =levels)
        geojson = geojsoncontour.contourf_to_geojson(
            contourf=contour,
            ndigits=3,
            unit='mm'
        )
        plt.close()
        return geojson

city_code = "331000"
start = '2023-07-01 20:52:00'
end = '2023-07-02 20:52:00'
worker = station_text(city_code,start,end)
plot_data = "none"
# worker.text_wind()
# worker.text_view()
# worker.text_tmax()
# worker.text_rain()
worker.main()

# 画图
# plot_type = "rain"
# orig = worker.data
# data = orig.sort_values(by="rain",ascending=False)
# plot = data[(data['rain']>0) & (data['rain']<5009)]
# plot_data = plot.to_json(orient='records',force_ascii=False)
# df = pd.read_json(plot_data, orient='records')

# worker.plot_rain()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 495 ms, sys: 0 ns, total: 495 ms
Wall time: 486 ms


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

('【雨情通报】 ：全市出现小到中雨。面雨量：5.11毫米，各市（县）面雨量如下:天台县:11.08毫米,仙居县:4.89毫米,临海市:1.69毫米,椒江区:1.18毫米,三门县:0.25毫米,黄岩区:0.13毫米,温岭市:0.1毫米。单站雨量较大的有：天台县溪地备份56.7毫米，天台县溪地52.3毫米，天台县龙溪46.0毫米，天台县里石门35.7毫米，天台县后岸34.8毫米，天台县田芯34.3毫米，天台县茶坑32.7毫米，天台县紫凝山29.8毫米，仙居县白塔21.9毫米，仙居县厚仁村21.1毫米。其中，≥50毫米的有2站,≥30毫米的有5站,≥10毫米的有30站。乡镇雨量较大的有：天台县溪地备份56.7毫米，天台县溪地52.3毫米，天台县龙溪46.0毫米，天台县里石门35.7毫米，天台县后岸34.8毫米，天台县田芯34.3毫米，天台县茶坑32.7毫米，天台县紫凝山29.8毫米，仙居县白塔21.9毫米，仙居县厚仁村21.1毫米。其中，≥50毫米的有1站,≥30毫米的有1站,≥10毫米的有13站。<br>【风力通报】 全市出现10级大风，风力较大的有：天台县下陈胡26.4米/秒，临海市雀儿岙22.5米/秒，路桥区白果22.3米/秒，临海市东矶21.0米/秒，温岭市北港20.5米/秒。其中，10级风有1站,9级风有3站,8级风有10站,7级风有43站,6级风有66站,6级风以下有18站。<br>【高温通报】 全市出现40℃以上的高温，温度较高的有：仙居县下岸水库40.0℃，仙居县横溪39.7℃，天台县西方洋39.4℃，仙居县万竹口39.3℃，仙居县工业园区38.9℃。其中，40度以上的有1站,37度以上的有76站。<br>【能见度】 全市出现低于50米的强浓雾，能见度较低的有：天台县左溪能见度210米，黄岩区上垟281米，玉环市披山308米，天台县西方洋313米，椒江区一江码头379米。其中，强浓雾的有18站,大雾的有6站。',
 '[{"Province":"浙江省","City":"台州市","Cnty":"天台县","Town":"街头镇","Lat":29.0714,"Lon":120.8411,"Height":133.0,"Stationlevl":14,"StationName":"溪地备份","IIIII":"K8771","tmax":35.6,"tm

,Unnamed: 0,Province,City,Cnty,Town,Lat,Lon,Height,Stationlevl,StationName,IIIII,tmax,tmin,rain,rainhour,view,wind,windave
0,158,浙江省,台州市,天台县,街头镇,29.0714,120.8411,133.0,14,溪地备份,K8771,35.6,24.0,56.7,38.3999,999999,999999.0,7.0
1,167,浙江省,台州市,天台县,街头镇,29.0714,120.8411,133.0,14,溪地,K8740,35.5,24.0,52.3,34.7000,999999,11.9,7.0
2,329,浙江省,台州市,天台县,龙溪乡,29.0517,120.7939,123.0,16,龙溪,K8705,35.3,24.4,46.0,22.0000,999999,9.3,5.3
3,512,浙江省,台州市,天台县,街头镇,29.0728,120.7303,248.0,14,里石门,K8726,34.9,24.1,35.7,34.4000,999999,9.1,6.6
4,187,浙江省,台州市,天台县,街头镇,29.0786,120.8139,100.0,14,后岸,K8748,34.4,24.9,34.8,14.4000,999999,14.9,10.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,198,浙江省,台州市,黄岩区,宁溪镇,28.5917,120.9753,150.0,14,宁溪备份,K8202,37.2,25.5,0.1,0.1000,999999,7.0,4.5
204,490,浙江省,台州市,黄岩区,平田乡,28.5381,121.0739,393.0,14,平田,K8221,32.9,23.7,0.1,0.1000,999999,9.3,5.6
205,302,浙江省,台州市,临海市,邵家渡街道,28.8400,121.2647,14.0,14,牛头山,K8608,36.3,24.6,0.1,0.1000,999999,8.0,4.9
206,90,浙江省,台州市,临海市,白水洋镇,29.0000,120.9667,804.0,14,安基山,K8643,30.2,23.0,0.1,0.1000,999999,9.6,7.1


In [12]:
import numpy as np
import pandas as pd 

import pandas as pd
df = worker.rain_std()
def my_sq(col):
    lat = col['Lat']
    lon = col['Lon']
    rain = col['rain']
    IIIII = col['IIIII']
    s = lat - 0.07
    n = lat + 0.07
    w = lon - 0.07
    e = lon + 0.07
    stdf = df[(df['Lat']>s)&(df['Lat']<n)&(df['Lon']<e)&(df['Lon']>w)]
    stdq = stdf[(True^stdf['IIIII'].isin([IIIII]))]
    stdmean = stdq['rain'].mean()
    stds = stdq['rain'].std()*3
    stdabs = abs(rain - stdmean)
    if stdabs > stds:
        #value = stdabs - stds
        value = 1
    else:
        #value = stdabs - stds
        value = 0
    del stdf,stdq
    return value
df['std'] = df.apply(my_sq,axis=1) 
df[df['IIIII']=='K8646']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,Province,City,Cnty,Town,Lat,Lon,Height,Stationlevl,StationName,IIIII,tmax,tmin,rain,rainhour,view,wind,windave,std
431,431,浙江省,台州市,临海市,邵家渡街道,28.8483,121.3367,60.0,14,南岙,K8646,35.2,24.2,7.8,8.9,999999,5.7,3.2,1


In [13]:
df[df['std']>0]

,Unnamed: 0,Province,City,Cnty,Town,Lat,Lon,Height,Stationlevl,StationName,IIIII,tmax,tmin,rain,rainhour,view,wind,windave,std
84,84,浙江省,台州市,临海市,永丰镇,28.9672,121.0675,20.0,14,沿岸,K8663,37.1,25.4,2.7,2.6,999999,3.5,2.6,1
315,315,浙江省,台州市,椒江区,章安街道,28.7539,121.4047,21.0,14,马峙,K8104,35.3,26.5,3.7,3.5,999999,9.1,5.4,1
402,402,浙江省,台州市,三门县,花桥镇,28.9208,121.4714,21.0,14,花桥,K8809,36.4,26.2,0.7,0.6,999999,7.7,5.1,1
414,414,浙江省,台州市,黄岩区,上郑乡,28.6056,120.9439,99.0,14,下郑,K8299,37.3,25.9,0.6,0.2,999999,999999.0,999999.0,1
431,431,浙江省,台州市,临海市,邵家渡街道,28.8483,121.3367,60.0,14,南岙,K8646,35.2,24.2,7.8,8.9,999999,5.7,3.2,1
492,492,浙江省,台州市,临海市,古城街道,28.8717,121.0797,11.0,14,两桥,K8622,35.6,26.5,17.3,9.9,999999,7.0,5.6,1


In [35]:
df[(True^df['IIIII'].isin(['K8646']))]

,Unnamed: 0,Province,City,Cnty,Town,Lat,Lon,Height,Stationlevl,StationName,IIIII,tmax,tmin,rain,rainhour,view,wind,windave,std
4,4,浙江省,台州市,临海市,尤溪镇,28.7425,121.1011,17.0,14,尤溪,K8606,37.6,26.1,0.1,0.1000,999999,6.6,3.4,-0.014772
6,6,浙江省,台州市,天台县,泳溪乡,29.1639,121.3750,235.0,14,大柳溪,K8798,35.4,23.5,4.1,2.3000,999999,999999.0,999999.0,NaN
11,11,浙江省,台州市,黄岩区,茅畲乡,28.5950,121.1314,29.0,14,茅畲,K8216,36.4,25.1,0.1,0.1000,999999,7.9,6.0,-0.025000
12,12,浙江省,台州市,天台县,洪畴镇,29.0997,121.2433,409.0,14,东安隐,K8766,33.1,24.3,0.6,0.6000,999999,999999.0,999999.0,-2.790364
15,15,浙江省,台州市,仙居县,白塔镇,28.7256,120.5786,106.0,14,白塔,K8901,36.7,25.0,21.9,15.9999,999999,10.1,7.1,2.704324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,575,浙江省,台州市,仙居县,淡竹乡,28.6719,120.5706,105.0,14,淡竹,K8911,36.6,25.0,3.1,2.9999,999999,7.9,5.3,-4.162336
576,576,浙江省,台州市,临海市,括苍镇,28.8572,120.9875,36.0,14,城西中学,K8657,37.6,26.4,0.1,0.1000,999999,4.1,2.7,-0.685236
578,578,浙江省,台州市,仙居县,双庙乡,28.7889,120.8575,92.0,14,双庙,K8906,37.9,24.9,3.8,3.7999,999999,0.0,0.0,0.771929
581,581,浙江省,台州市,三门县,亭旁镇,29.0025,121.3147,138.0,16,佃石,K8818,36.5,25.6,0.1,0.1000,999999,999999.0,4.9,0.000000


In [29]:
df[df['IIIII']=='K8646']

,Unnamed: 0,Province,City,Cnty,Town,Lat,Lon,Height,Stationlevl,StationName,IIIII,tmax,tmin,rain,rainhour,view,wind,windave,std
431,431,浙江省,台州市,临海市,邵家渡街道,28.8483,121.3367,60.0,14,南岙,K8646,35.2,24.2,7.8,8.9,999999,5.7,3.2,2.4


In [ ]:
df = DataFrame(np.random.randn(10000,3))
df.abs()  # 每个元素绝对值
# 判断异常数据.
# 去除掉所有满足以下情况的行
# 大于三倍标准偏差
cond = (df.abs() > df.std()*3).any(axis=1)
df[cond]   # 存在大于三倍标准偏差元素的异常行
df[~cond]  # 取反, 去掉异常的行

In [ ]:
data.to_json(orient='records',force_ascii=False).txt

In [10]:
d = {'a': 1, 'b': 3}
# 遍历d时,得到的是字典的key,然后将key对应的value设置为比较的对象
res = max(d, key=lambda x: d[x])
print(res)  # b

b


In [ ]:
1、小雨：降雨量小于10mm者。

2、中雨：降雨量10～25mm者。

3、大雨：降雨量25～50mm者。

4、暴雨：降雨量50～100mm者。

5、大暴雨：降雨量100～250mm者。

6、特大暴雨：1d（或24h）降雨量在250mm以上者。


小到中雨：

是指在24小时内降雨量在5~16.9毫米之间。


中到大雨：

是指在24小时内降雨量在17~37.9毫米之间。


大到暴雨：

是指在24小时内降雨量在33~74.9毫米之间。


阵雨：

阵雨是雨时短促、开始和终止都很突然，降水强度变化很大的雨，有时伴有闪电和雷鸣。阵雨有不连续的特点，范围一般不大，带有雨幡，且往往伴有大风出现。


雷阵雨：

雷阵雨要比阵雨要剧烈得多，还伴有放电现象，雷阵雨来临之前，往往会出现狂风大作、雷雨交加的天气现象。

0 - 10 
10 - 25
25 -50 
50 - 100
100 - 250 


# 并行计算的算法

In [68]:
%%time
import time
from joblib import Parallel, delayed

def square(num):
    if num==1:
        time.sleep(1)  # 模拟耗时的计算操作
    elif num==2:
        time.sleep(2)
    elif num==3:
        time.sleep(3) 
    elif num==4:
        time.sleep(4)
    elif num==5:
        time.sleep(5)
    elif num==6:
        time.sleep(6)
    return num ** 2

if __name__ == '__main__':
    numbers = [1,2,3,4,5,6]
    start_time = time.time()
    # 并行计算每个数字的平方
    results = Parallel(n_jobs=1)(delayed(square)(num) for num in numbers)
    end_time = time.time()
    # 打印计算结果
    print(results)
    print("并行处理时间：", end_time - start_time)

[1, 4, 9, 16, 25, 36]
并行处理时间： 21.022823333740234
CPU times: user 3.14 ms, sys: 1.3 ms, total: 4.44 ms
Wall time: 21 s


# 备份

In [ ]:
%%time
# coding=utf8
import pymysql
import pymssql 
import numpy as np
import pandas as pd
import pickle
import redis
import datetime as dtt
# from cma.music.DataQueryClient import DataQueryClient
import json

class station_text:
    def __init__(self,city_code,start,end):
        #self.rs = redis.Redis(host='127.0.0.1', port=6379)
#         self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="tzqxj58660",db="ZJSZDZDB")
        self.userId = "BEHZ_TZSJ_TZSERVICE" 
        self.pwd = "Liyuan3970!@" 
        self.dataFormat = "json"
        self.city_code = city_code
        self.start = start
        self.end = end
        self.data = self.comupt_city_csv(city_code,start,end)
        self.city_codes = ["331000"]
    def comput_city(self,city_code,start,end):
        """快报或者统计数据的接口"""
        # getSurfEleInRegionByTime
        date_start = dtt.datetime.strptime(start,'%Y-%m-%d %H:%M:%S') 
        date_end = dtt.datetime.strptime(end,'%Y-%m-%d %H:%M:%S') 
        offset = dtt.timedelta(minutes=-60*8)
        label_start = (date_start + offset).strftime('%Y%m%d%H%M') + "00"
        label_end = (date_end + offset).strftime('%Y%m%d%H%M') + "00"   
        labels = "[" + label_start + "," + label_end + "]"
        #timelabel = self.decode_time_str(timesdelay)
        client = DataQueryClient()
        interfaceId = "statSurfEleInRegion"  
        params = {
            'dataCode':"SURF_CHN_MUL_MIN",  #SURF_CHN_MUL_HOR
            'adminCodes':"331000",
            'elements':"Province,City,Cnty,Town,Lat,Lon,Alti,Station_levl,Station_Name,Station_Id_C,",
            'timeRange':labels,
            'statEles':'MAX_TEM,MIN_TEM,SUM_PRE,MAX_PRE_1h,MIN_VIS_HOR_1MI,MAX_WIN_S_Gust_Max,MAX_WIN_S_Avg_2mi',
            #'orderBy':"MAX_WIN_S_Inst_Max:desc",
            'staLevels':"011,012,013,014,015,016", # 12国家站 14区域站
            'limitCnt':"100000000"
        }
        result = client.callAPI_to_serializedStr(self.userId, self.pwd, interfaceId, params, self.dataFormat)
        #rint(result)
        result_json = json.loads(result)
        clomns = "Province,City,Cnty,Town,Lat,Lon,Alti,Station_levl,Station_Name,Station_Id_C,MAX_TEM,MIN_TEM,SUM_PRE,MAX_PRE_1h,MIN_VIS_HOR_1MI,MAX_WIN_S_Gust_Max,MAX_WIN_S_Avg_2mi".split(",")
        data = pd.DataFrame(result_json['DS'],columns=clomns)
        data.columns = "Province,City,Cnty,Town,Lat,Lon,Height,Stationlevl,StationName,IIIII,tmax,tmin,rain,rainhour,view,wind,windave".split(",")
        return data
    def comupt_city_csv(self,city_code,start,end):
#         data = pd.read_csv("static/data/downfile/comput.csv")
        data = pd.read_csv("downfile/comput.csv")
        return data
    def text_wind(self):
        orig = self.data
        data = orig.sort_values(by="wind",ascending=False)#.head(5).to_dict()
        ## 数据分级
        bins=[0,10.7,13.8,17.1,20.7,24.4,28.4,32.6,36.9,41.4,46.1,50.9,56,80]
        labels=['6级风以下','6级风','7级风','8级风','9级风','10级风','11级风','12级风','13级风','14级风','15级风','16级风','17级风']
        wind = data[(data['wind']>10.7) & (data['wind']<5009)]
        wind['rank']=pd.cut(wind['wind'],bins,right=False,labels=labels)
        del wind['Unnamed: 0'] # 天擎不需要
        wind.reset_index(drop=True)# 天擎不需要
        # 获取单站较大的有
        testframe = wind.head(5)
        wind_json = wind.sort_values(by="wind",ascending=False).to_json(orient='records',force_ascii=False) 
        indextext = ""
        for index,row in testframe.iterrows():
            if self.city_code in self.city_codes:
                indextext = indextext + row['Cnty'] + row['StationName'] + str(row['wind']) + "米/秒，"
                #print(index,row['City'],row['Cnty'],row['Town'],row['wind'])
            else:
                indextext = indextext + row['Town'] + row['StationName'] + str(row['wind']) + "米/秒，"
                #print(index,row['City'],row['Cnty'],row['Town'],row['wind'])
        indextext = indextext[:-1] + "。"
        rank_station = {
            '17级及以上风':len(wind[(wind['wind']>=56.4) & (wind['wind']<80)].value_counts()),
            '16级风':len(wind[(wind['wind']>=50.9) & (wind['wind']<56.4)].value_counts()),
            '15级风':len(wind[(wind['wind']>=46.1) & (wind['wind']<50.9)].value_counts()),
            '14级风':len(wind[(wind['wind']>=41.4) & (wind['wind']<46.1)].value_counts()),
            '13级风':len(wind[(wind['wind']>=36.9) & (wind['wind']<41.4)].value_counts()),
            '12级风':len(wind[(wind['wind']>=32.6) & (wind['wind']<36.9)].value_counts()),
            '11级风':len(wind[(wind['wind']>=28.4) & (wind['wind']<32.6)].value_counts()),
            '10级风':len(wind[(wind['wind']>=24.4) & (wind['wind']<28.4)].value_counts()),
            '9级风':len(wind[(wind['wind']>=20.7) & (wind['wind']<24.4)].value_counts()),
            '8级风':len(wind[(wind['wind']>=17.1) & (wind['wind']<20.7)].value_counts()),
            '7级风':len(wind[(wind['wind']>=13.8) & (wind['wind']<17.1)].value_counts()),
            '6级风':len(wind[(wind['wind']>=10.7) & (wind['wind']<13.8)].value_counts()),
            '6级风以下':len(wind[(wind['wind']>0) & (wind['wind']<10.7)].to_dict())
        }
        #  统计各等级风力的个数
        numbertext = ""
        for key, value in rank_station.items():
            if value>0:
                numbertext = numbertext + key +"有" + str(value) + "站,"
        numbertext = numbertext[:-1] + "。"
        if rank_station['17级及以上风'] >0 :
            text = "【风力通报】 全市出现17级及以上大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['16级风'] >0 :
            text = "【风力通报】 全市出现16级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext  
        elif rank_station['15级风'] >0 :
            text = "【风力通报】 全市出现15级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['14级风'] >0 :
            text = "【风力通报】 全市出现14级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['13级风'] >0 :
            text = "【风力通报】 全市出现13级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['12级风'] >0 :
            text = "【风力通报】 全市出现12级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['11级风'] >0 :
            text = "【风力通报】 全市出现11级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['10级风'] >0 :
            text = "【风力通报】 全市出现10级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['9级风'] >0 :
            text = "【风力通报】 全市出现7～9级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['8级风'] >0 :
            text = "【风力通报】 全市出现6～8级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        else:
            text = ""
        return text,wind_json
    def text_view(self):
        orig = self.data
        data = orig.sort_values(by="view",ascending=True)#.head(5).to_dict()
        bins=[0,50,200,500,99999]
        labels=['强浓雾','浓雾','大雾','正常']
        view = data[(data['view']>0) & (data['view']<1000)]
        view['rank']=pd.cut(view['view'],bins,right=False,labels=labels)
        del view['Unnamed: 0'] # 天擎不需要
        view.reset_index(drop=True)# 天擎不需要
        rank_station = {
           '强浓雾':len(view[(view['view']>0) & (view['view']<50)].to_dict()),
            '浓雾':len(view[(view['view']>=50) & (view['view']<200)].value_counts()),
            '大雾':len(view[(view['view']>=200) & (view['view']<500)].value_counts()),
            '正常':len(view[(view['view']>=500) & (view['view']<99990)].value_counts()) 
        }
        # 获取单站较大的有
        testframe = view.head(5) 
        view_json = view.sort_values(by="view",ascending=True).to_json(orient='records',force_ascii=False)
        indextext = ""
        for index,row in testframe.iterrows():
            if self.city_code in self.city_codes:
                indextext = indextext + row['Cnty'] + row['StationName'] + str(row['view']) + "米，"
            else:
                indextext = indextext + row['Town'] + row['StationName'] + str(row['view']) + "米，"
        indextext = indextext[:-1] + "。"
        #  统计各等级风力的个数
        numbertext = ""
        for key, value in rank_station.items():
            if value>0 and key!="正常":
                numbertext = numbertext + key +"的有" + str(value) + "站,"
        numbertext = numbertext[:-1] + "。"
        if rank_station['强浓雾'] >0 :
            text = "【能见度】 全市出现低于50米的强浓雾，能见度较低的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['浓雾'] >0 :
            text = "【能见度】 全市出现低于200米的浓雾，能见度较低的有："
            text = text + indextext + "其中，"  + numbertext  
        elif rank_station['大雾'] >0 :
            text = "【能见度】 全市出现低于500米的大雾，能见度较低的有："
            text = text + indextext + "其中，"  + numbertext
        else:
            text =""
        return text,view_json
    def text_tmax(self):
        orig = self.data
        data = orig.sort_values(by="tmax",ascending=False)#.head(5).to_dict()
        bins=[-40,35,37,40,80]
        labels=['正常','35度以上','37度以上','40度以上']
        tmax = data[(data['tmax']>-40) & (data['tmax']<100)]
        tmax['rank']=pd.cut(tmax['tmax'],bins,right=True,labels=labels)
        del tmax['Unnamed: 0'] # 天擎不需要
        tmax.reset_index(drop=True)# 天擎不需要
        rank_station = {
            '40度以上':len(tmax[(tmax['tmax']>=40) & (tmax['tmax']<80)].value_counts()),
            '37度以上':len(tmax[(tmax['tmax']>=37) & (tmax['tmax']<40)].value_counts()),
            '35度以上':len(tmax[(tmax['tmax']>=35) & (tmax['tmax']<37)].value_counts()),
           '正常':len(tmax[(tmax['tmax']>-40) & (tmax['tmax']<35)].to_dict())      
        }
        # 获取单站较大的有
        testframe = tmax.head(5) 
        tmax_json = tmax.sort_values(by="tmax",ascending=False).to_json(orient='records',force_ascii=False)
        indextext = ""
        for index,row in testframe.iterrows():
            if self.city_code in self.city_codes:
                indextext = indextext + row['Cnty'] + row['StationName'] + str(row['tmax']) + "℃，"
            else:
                indextext = indextext + row['Town'] + row['StationName'] + str(row['tmin']) + "℃，"
        indextext = indextext[:-1] + "。"
        #  统计各等级风力的个数
        numbertext = ""
        for key, value in rank_station.items():
            if value>0 and key not in ["正常","35度以上"]:
                numbertext = numbertext + key +"的有" + str(value) + "站,"
        numbertext = numbertext[:-1] + "。"
        if rank_station['40度以上'] >0 :
            text = "【高温通报】 全市出现40℃以上的高温，温度较高的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['37度以上'] >0 :
            text = "【高温通报】 全市出现40℃以上的高温，温度较高的有："
            text = text + indextext + "其中，"  + numbertext  
        elif rank_station['35度以上'] >0 :
            text = "【高温通报】 全市出现40℃以上的高温，温度较高的有："
            text = text + indextext 
        else:
            text = ""
        return text,tmax_json
    def text_rain(self):
        # 面雨量
        orig = self.data
        data = orig.sort_values(by="rain",ascending=False)
        bins=[0,30,50,80,100,150,200,250,300,400,500,800,1000,2000]
        labels=['≥0毫米','≥30毫米','≥50毫米','≥80毫米','≥100毫米','≥150毫米','≥200毫米','≥250毫米','≥300毫米','≥400毫米','≥500毫米','≥800毫米','≥1000毫米']
        bins_text = [0,5,16.9,37.9,74.5,5000]
        labels_text = ['小雨','小到中雨','中到大雨','大到暴雨','大暴雨']
        rain = data[(data['rain']>0) & (data['rain']<5009)]
        rain['rank']=pd.cut(rain['rain'],bins,right=False,labels=labels)
        rain['rank_label']=pd.cut(rain['rain'],bins_text,right=False,labels=labels_text)
        del rain['Unnamed: 0'] # 天擎不需要
        rain.reset_index(drop=True)
        # 数据校验
        def vaild_rain(col):
            lat = col['Lat']
            lon = col['Lon']
            pre = col['rain']
            IIIII = col['IIIII']
            s = lat - 0.07
            n = lat + 0.07
            w = lon - 0.07
            e = lon + 0.07
            stdf = rain[(rain['Lat']>s)&(rain['Lat']<n)&(rain['Lon']<e)&(rain['Lon']>w)]
            stdq = stdf[(True^stdf['IIIII'].isin([IIIII]))]
            stdmean = stdq['rain'].mean()
            stds = stdq['rain'].std()*3
            stdabs = abs(pre - stdmean)
            if stdabs > stds:
                #value = stdabs - stds
                value = 1
            else:
                #value = stdabs - stds
                value = 0
            del stdf,stdq
            return value
        rain['std'] = rain.apply(vaild_rain,axis=1) 
        # 面雨量
        if self.city_code in self.city_codes:
            cnty = rain.groupby(['Cnty'])['rain'].mean().to_dict()
            
            text_average_city = "面雨量：" + str(round(rain['rain'].mean(),2)) + "毫米，" +"各市（县）面雨量如下:"
        else:
            cnty = rain.groupby(['Town'])['rain'].mean().to_dict()
            text_average_city = "各乡镇面雨量如下:"
        items = sorted(cnty.items())
        sorted_cnty = {k: v for k, v in sorted(cnty.items(), key=lambda x: x[1], reverse=True)}
        for key, value in sorted_cnty.items():
            text_average_city = text_average_city + key + ":" + str(round(value,2)) + "毫米,"
        text_average_city = text_average_city[:-1] + "。"    
        # 单站前十
        rain_max = rain.sort_values(by="rain",ascending=False).head(10)
        rain_json = rain.sort_values(by="rain",ascending=False).to_json(orient='records',force_ascii=False)
        indextext = "单站雨量较大的有："
        for index,row in rain_max.iterrows():
            if self.city_code in self.city_codes:
                indextext = indextext + row['Cnty'] + row['StationName'] + str(row['rain']) + "毫米，"
            else:
                indextext = indextext + row['Town'] + row['StationName'] + str(row['rain']) + "毫米，"
        indextext = indextext[:-1] + "。"
        # 乡镇前十
        town_max = rain.groupby(['Town','Cnty'])['rain'].max().sort_values(ascending=False).head(10)
        indextext_town = "乡镇雨量较大的有："
        for index,row in rain_max.iterrows():
            if self.city_code in self.city_codes:
                indextext_town = indextext_town + row['Cnty'] + row['StationName'] + str(row['rain']) + "毫米，"
            else:
                indextext_town = indextext_town + row['Town'] + row['StationName'] + str(row['rain']) + "毫米，"
        indextext_town = indextext_town[:-1] + "。"
        # 雨量个数统计
        rank_station = {
            '≥1000毫米':len(rain[(rain['rain']>=1000) & (rain['rain']<2000)].value_counts()),
            '≥800毫米':len(rain[(rain['rain']>=800) & (rain['rain']<1000)].value_counts()),
            '≥500毫米':len(rain[(rain['rain']>=500) & (rain['rain']<800)].value_counts()),
            '≥400毫米':len(rain[(rain['rain']>=400) & (rain['rain']<500)].value_counts()),
            '≥300毫米':len(rain[(rain['rain']>=300) & (rain['rain']<400)].value_counts()),
            '≥250毫米':len(rain[(rain['rain']>=250) & (rain['rain']<300)].value_counts()),
            '≥200毫米':len(rain[(rain['rain']>=200) & (rain['rain']<250)].value_counts()),
            '≥150毫米':len(rain[(rain['rain']>=150) & (rain['rain']<200)].value_counts()),
            '≥100毫米':len(rain[(rain['rain']>=100) & (rain['rain']<150)].value_counts()),
            '≥80毫米':len(rain[(rain['rain']>=80) & (rain['rain']<100)].value_counts()),
            '≥50毫米':len(rain[(rain['rain']>=50) & (rain['rain']<80)].value_counts()),
            '≥30毫米':len(rain[(rain['rain']>=30) & (rain['rain']<50)].value_counts()),
            '≥10毫米':len(rain[(rain['rain']>=10) & (rain['rain']<30)].value_counts()),
            '≥0毫米':len(rain[(rain['rain']>0) & (rain['rain']<10)].value_counts()),
        }
        numbertext = "其中，"
        for key, value in rank_station.items():
            if value>0 and key not in ["≥0毫米"]:
                numbertext = numbertext + key +"的有" + str(value) + "站,"
        numbertext = numbertext[:-1] + "。"
        # 雨量乡镇数统计
        town_group = rain.groupby(['Town'])['rain'].max()  
        dict_town = {'Town':town_group.index,'rain':town_group.values}
        town_range = pd.DataFrame(data =dict_town)
        town_range['rank']=pd.cut(town_range['rain'],bins,right=False,labels=labels)
        rank_station_town = {
            '≥1000毫米':len(town_range[(town_range['rain']>=1000) & (town_range['rain']<2000)].value_counts()),
            '≥800毫米':len(town_range[(town_range['rain']>=800) & (town_range['rain']<1000)].value_counts()),
            '≥500毫米':len(town_range[(town_range['rain']>=500) & (town_range['rain']<800)].value_counts()),
            '≥400毫米':len(town_range[(town_range['rain']>=400) & (town_range['rain']<500)].value_counts()),
            '≥300毫米':len(town_range[(town_range['rain']>=300) & (town_range['rain']<400)].value_counts()),
            '≥250毫米':len(town_range[(town_range['rain']>=250) & (town_range['rain']<300)].value_counts()),
            '≥200毫米':len(town_range[(town_range['rain']>=200) & (town_range['rain']<250)].value_counts()),
            '≥150毫米':len(town_range[(town_range['rain']>=150) & (town_range['rain']<200)].value_counts()),
            '≥100毫米':len(town_range[(town_range['rain']>=100) & (town_range['rain']<150)].value_counts()),
            '≥80毫米':len(town_range[(town_range['rain']>=80) & (town_range['rain']<100)].value_counts()),
            '≥50毫米':len(town_range[(town_range['rain']>=50) & (town_range['rain']<80)].value_counts()),
            '≥30毫米':len(town_range[(town_range['rain']>=30) & (town_range['rain']<50)].value_counts()),
            '≥10毫米':len(town_range[(town_range['rain']>=10) & (town_range['rain']<30)].value_counts()),
            '≥0毫米':len(town_range[(town_range['rain']>0) & (town_range['rain']<10)].value_counts()),
        }
        numbertext_town = "其中，"
        for key, value in rank_station_town.items():
            if value>0 and key not in ["≥0毫米"]:
                numbertext_town = numbertext_town + key +"的有" + str(value) + "站,"
        numbertext_town = numbertext_town[:-1] + "。"
        # 统计全市雨情
        rank_text = {
            '大暴雨':sum(rain[rain['rank_label']=='大暴雨']['rain']),
            '大到暴雨':sum(rain[rain['rank_label']=='大到暴雨']['rain']),
            '中到大雨':sum(rain[rain['rank_label']=='中到大雨']['rain']),
            '小到中雨':sum(rain[rain['rank_label']=='小到中雨']['rain']),
            '小雨':sum(rain[rain['rank_label']=='小雨']['rain'])
        }
        res = max(rank_text, key=lambda x: rank_text[x])
        if max(rank_text, key=lambda x: rank_text[x])=="大暴雨":
            text = "【雨情通报】 ：全市出现" + res +"。" + text_average_city + indextext + numbertext + indextext_town + numbertext_town
        elif max(rank_text, key=lambda x: rank_text[x])=="大到暴雨":
            text = "【雨情通报】 ：全市出现" + res +"。" + text_average_city + indextext + numbertext + indextext_town + numbertext_town
        elif max(rank_text, key=lambda x: rank_text[x])=="中到大雨":
            text = "【雨情通报】 ：全市出现" + res +"。" + text_average_city + indextext + numbertext + indextext_town + numbertext_town
        elif max(rank_text, key=lambda x: rank_text[x])=="小到中雨":
            text = "【雨情通报】 ：全市出现" + res +"。" + text_average_city + indextext + numbertext + indextext_town + numbertext_town
        elif max(rank_text, key=lambda x: rank_text[x])=="小雨":
            text = "【雨情通报】 ：全市出现" + res +"。" + text_average_city + indextext + numbertext + indextext_town + numbertext_town
        return text,rain_json
    def main(self):
        text = ""
        text_rain,rain_json = self.text_rain()
        text_wind,wind_json = self.text_wind()
        text_tmax,tmax_json = self.text_tmax()
        text_view,view_json = self.text_view()
        text = text + text_rain + "<br>" + text_wind + "<br>" + text_tmax + "<br>" + text_view
        # text = text + text_rain + "\n" + text_wind + "/n" + text_tmax + "/n" + text_view
        return text,rain_json,wind_json,tmax_json,view_json
    def plot_rain(self,plot_type,plot_data):
        if plot_type =="none":
            orig = self.data
            data = orig.sort_values(by="rain",ascending=False)
            rain = data[(data['rain']>0) & (data['rain']<5009)]
        else:
            rain = pd.read_json(plot_data, orient='records')    
        lat = np.array(rain['Lat'].to_list())
        lon = np.array(rain['Lon'].to_list())
        Zi = np.array(rain['rain'].to_list())
        data_max = max(Zi)
        data_min = min(Zi)
        np.set_printoptions(precision = 2)
        x = np.arange(120.0,122.0,0.015)
        y = np.arange(27.8,29.5,0.015)
        nx0 =len(x)
        ny0 =len(y)
        X, Y = np.meshgrid(x, y)#100*100
        P = np.array([X.flatten(), Y.flatten() ]).transpose()    
        Pi =  np.array([lon, lat ]).transpose()
        Z_linear = griddata(Pi, Zi, P, method = "nearest").reshape([ny0,nx0])
        gauss_kernel = Gaussian2DKernel(0.1)
        smoothed_data_gauss = convolve(Z_linear, gauss_kernel)
        data_xr = xr.DataArray(Z_linear, coords=[ y,x], dims=["lat", "lon"])
        lat = data_xr.lat
        lon = data_xr.lon
        lons, lats = np.meshgrid(lon, lat)
        colorslist = ['#FFFFFF','#A6F28f','#3DBA3D',"#61B8FF","#0000E1","#FA00FA","#800040"]# 24降水
        levels = [0,1,10,25,50,100,250,1000]
        cmaps = LinearSegmentedColormap.from_list('mylist',colorslist,N=7)
        cmap_nonlin = nlcmap(cmaps, levels)
        contour = plt.contourf(lons,lats,data_xr,cmap=cmap_nonlin,levels =levels)
        geojson = geojsoncontour.contourf_to_geojson(
            contourf=contour,
            ndigits=3,
            unit='mm'
        )
        plt.close()
        with open("shpfile/country/shp/tiantai.json", "r", encoding="utf-8") as f:
            shpjson = json.load(f)
        return shpjson,geojson